<a href="https://colab.research.google.com/github/rlewa/projetofutebol/blob/main/notebooks/data/Scraping6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+'https://github.com/rlewa/projetofutebol'
from packages.packages import *
from packages.functions import *
clear_output()

Scraping news from the website: https://www.thesun.co.uk/sport/football/

In [ ]:
def get_teams_links_thesun(URL: str = 'https://www.thesun.co.uk/sport/football/premierleague/') -> dict:

  '''
    Gets the links for each team available on 'thesun.co.uk' website

    Input:
      str: website link
      bool: pl_teams that is True for only Premier League teams and False for any team 

    Return:
      dict: dict with all teams names and links available
  '''

  req = requests.get(URL)
  soup = BeautifulSoup(req.text, 'html.parser')

  teams_links = np.array([])

  for link in soup.find_all('a', class_ = "theme__before-color"):
    teams_links = np.append(teams_links, str(link).split('<a class="theme__before-color" href="')[1].split('"></a>')[0])
  
  teams_links = teams_links[:20]

  return teams_links

In [ ]:
team_links = get_teams_links_thesun()
team_links

array(['https://www.thesun.co.uk/sport/football/team/1196653/arsenal/',
       'https://www.thesun.co.uk/sport/football/team/1196670/aston-villa/',
       'https://www.thesun.co.uk/sport/football/team/1225644/brentford/',
       'https://www.thesun.co.uk/sport/football/team/3571589/brighton-hove-albion/',
       'https://www.thesun.co.uk/sport/football/team/1225466/burnley/',
       'https://www.thesun.co.uk/sport/football/team/1196660/chelsea/',
       'https://www.thesun.co.uk/sport/football/team/1196666/crystal-palace/',
       'https://www.thesun.co.uk/sport/football/team/1196662/everton/',
       'https://www.thesun.co.uk/sport/football/team/1225859/leeds-united/',
       'https://www.thesun.co.uk/sport/football/team/1196651/leicester-city/',
       'https://www.thesun.co.uk/sport/football/team/1196659/liverpool/',
       'https://www.thesun.co.uk/sport/football/team/1196654/manchester-city/',
       'https://www.thesun.co.uk/sport/football/team/1196656/manchester-united/',
      

In [ ]:
def get_news_thesun(name):

  '''
    Gets all the news from the thesun.co.uk website: 'date', 'hour', title' and 'text'

    Input:
    Returns:
      bool: True if works
  '''

  standard_names = ['Arsenal FC', 'Aston Villa', 'Brentford FC', 'Brighton & Hove Albion', 'Burnley FC', 'Chelsea FC', 'Crystal Palace', 
                    'Everton FC', 'Leeds United', 'Leicester City', 'Liverpool FC', 'Manchester City', 'Manchester United', 'Newcastle United', 
                    'Norwich City', 'Southampton FC', 'Tottenham Hotspur', 'Watford FC', 'West Ham United', 'Wolverhampton Wanderers']

  teams_links = get_teams_links_thesun()
  
  if name == 'gabriel':
    teams_links = teams_links[:5]
    standard_names = ['Arsenal FC', 'Aston Villa', 'Brentford FC', 'Brighton & Hove Albion', 'Burnley FC']
  
  elif name == 'pet':
    teams_links = teams_links[5:7]
    standard_names = ['Chelsea FC', 'Crystal Palace']
  
  else:
    teams_links = teams_links[12:14]
    standard_names = ['Manchester United', 'Newcastle United']

  index = -1
  
  for team_link in teams_links:

    # index to change teams names
    index += 1

    print('\nTeam Name: ', standard_names[index])

    dict_news = {'date': [], 'hour': [], 'title': [], 'text': []}

    # Enter the first iteration of while.
    news_available = True 
    
    count_page = 1

    while news_available:
      try:

          string = team_link + "page/" + str(count_page) + "/"

          req  = requests.get(string) 

          if req.status_code == 200:
            
            soup = BeautifulSoup(req.text, 'html.parser')
            
            news_links = []
            
            # for each link, we get everything related to that news: 'date', 'title' and 'text'

            # find all 'a' in the class 'teaser-anchor', it represents all news links
            for a in soup.find_all('a', class_="teaser-anchor", href = True):
              link_text = str(a).split('href="')[1].split('"></a>')[0]
              
              if 'uncategorized' not in link_text:
                news_links.append(link_text)

            for link in news_links:
              
              #print(link)

              req = requests.get(link)
              soup_link = BeautifulSoup(req.text, 'html.parser')
              
              # only get news, not live blogs
              if soup_link.find(class_ = "text-slug t-p-background-color text-slug__live-blog") is None:
                
                # get main text 
                full_text = ' '
                for text in soup_link.find_all('div', class_="article__content"):
                  soup2 = BeautifulSoup(str(text), 'html.parser')
                  for p in soup2.find_all('p'):
                    if '<p class="has-text-align-center">' not in str(p) and '<strong>' not in str(p) and '<p>[video' not in str(p) and '<p dir="ltr"' not in str(p) and '<a href="https://twitter.com/goonerdave66"' not in str(p):
                      text = p.get_text()
                      text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#)|(\w+:\/\/\S+)", " ", text).split())
                      text = re.sub(r'pic.twitter.com/[\w]*',"", text)
                      full_text += ' ' + text + ' '

                full_text = full_text.strip()
                if len(full_text) == 0:
                  continue

                dict_news['text'].append(full_text)

                date_ = soup_link.find('span', class_ = "article__timestamp")

                # specific html tag when it gets the time instead of date
                if ":" in str(date_):
                  news_date = soup_link.find('div', class_ = "article__published")
                  news_hour = soup_link.find('span', class_ = "article__timestamp")
                  news_title = soup_link.find('section', class_ = "article__headline-section")
                  
                  if news_date is None:
                    dict_news['date'].append('')
                  dict_news['date'].append(news_date.get_text().replace(",", " ").strip()) 

                  if news_hour is None:
                    dict_news['hour'].append('')
                  dict_news['hour'].append(news_hour.get_text().strip() )

                  if news_title is None:
                    dict_news['title'].append('')
                  dict_news['title'].append(news_title.h1.text.replace("\n", ' ').replace("\t", ' ').strip())

                else:
                  news_title = soup_link.find('section', class_ = "article__headline-section")
                  news_hour = soup_link.find('span', class_ = "article__datestamp")
                  news_date = soup_link.find('span', class_ = "article__timestamp")

                  if news_date is None:
                    dict_news['date'].append('')
                  dict_news['date'].append(news_date.text.replace(',', '').strip()) 

                  if news_hour is None:
                    dict_news['hour'].append('')
                  dict_news['hour'].append(news_hour.text.replace(',', '').strip())

                  if news_title is None:
                    dict_news['title'].append('')
                  dict_news['title'].append(news_title.h1.text.replace("\n", ' ').replace("\t", ' ').strip())
                  
              else:
                continue

      except Exception as erro:

          if str(erro).lower() == 'list index out of range':
            break

          else:
            pass

      finally:
        
        if news_links == []:
          break

        if count_page % 5 == 0:
          print('Page number:', count_page, 'is OK!')        
        
        #Creates dataframe before takes the next team.
        df = pd.DataFrame(dict_news, columns=['date', 'hour', 'title', 'text'])  
        df['date'] = df['date'].apply(lambda x: x.split(' ')[0].replace('nd', '').replace('rd', '').replace('th', '').replace('st', '') + ' ' +
                                      x.split(' ')[1] + ' ' + x.split(' ')[2])

        df['date'] = df['date'].apply(lambda x: date.isoformat(dateutil.parser.parse(x).date())) 

        df.drop_duplicates(inplace = True)
    
        # Create path and save as csv.

        PATH = f'/content/drive/MyDrive/GitHub/Dados/Noticias/Scraping5/{standard_names[index]}.csv'
        df.to_csv(PATH, index = False)

        count_page += 1
        pass
      
  return True

In [ ]:
get_news_thesun('gabriel')

In [ ]:
#get_news_thesun('pet')

In [ ]:
#get_news_thesun('kaue')